<a href="https://colab.research.google.com/drive/1VsCiRxC4mUxOfr_7YPdIDItbTT0KaJJH?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>  

# Tune Llama 3 for text-to-SQL with Lamini Memory Tuning

In this notebook, you'll learn how to tune Llama 3 with Lamini Memory Tuning for a SQL LLM to remove hallucinations and lift accuracy from 30% to 95%.

You'll be using the `nba_roster` database, which contains information about NBA players, teams, and games. This database will serve as the foundation for your tuning process.

<div style="border: 2px solid #009fe3;  margin: 8px; padding: 16px; width: 80%;"> <b>NOTE</b> 

This notebook is an in-depth tutorial. Expected runtime for the notebook is ~ 6 minutes, but including full data generation and training the entire notebook can take several hours to run. Included in the notebook are several pre-prepared generated datasets and pre-prepared models for your convenience! Hang in there - it's totally worth it!
</div>


If you haven't already, please install `lamini` first!



In [2]:
%pip install lamini
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Auth

Before we begin, make sure to authenticate!

Please head over to https://app.lamini.ai/account to get your api key.
You can authenticate by writing the following to a file `~/.lamini/configure.yaml`

```python
production:
    key: <YOUR-LAMINI-API-KEY>
```
Alternatively, you can set your api key in this notebook by uncommenting `lamini.api_key = '<YOUR-LAMINI-API-KEY>'` and filling in your api key in the following cell before running!


In [3]:
import lamini 
# lamini.api_key = '<YOUR-LAMINI-API-KEY>'

In [4]:
import logging
import os
import random
from datetime import datetime
from pprint import pprint
from typing import AsyncIterator, Iterator, Union
import sqlite3
import copy
from tqdm import tqdm
from tabulate import tabulate

import pandas as pd
import jsonlines
from lamini.generation.base_prompt_object import PromptObject
from lamini.generation.generation_node import GenerationNode
from lamini.generation.base_prompt_object import PromptObject
from lamini.generation.generation_pipeline import GenerationPipeline
from util.get_schema import get_schema
from util.make_llama_3_prompt import make_llama_3_prompt
from util.setup_logging import setup_logging
from util.load_dataset import get_dataset
from util.get_default_finetune_args import get_default_finetune_args

logger = logging.getLogger(__name__)
engine = sqlite3.connect("./nba_roster.db")
setup_logging()

class Args:
    def __init__(self, 
                 max_examples=100, 
                 sql_model_name="meta-llama/Meta-Llama-3-8B-Instruct", 
                 gold_file_name="gold-test-set.jsonl",
                 training_file_name="generated_queries.jsonl",
                 num_to_generate=10):
        self.sql_model_name = sql_model_name
        self.max_examples = max_examples
        self.gold_file_name = gold_file_name
        self.training_file_name = training_file_name
        self.num_to_generate = num_to_generate

## Create a SQL Model with Llama 3 and Diagnose Hallucinations

First let's create a SQL LLM with Llama 3 and get a baseline. You can run the following python script which uses Llama 3.

In [5]:
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

question = """Who is the highest paid NBA player?"""
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""
prompt = make_llama_3_prompt(question, system)
print("Question:\n", question)

# Ask the model to generate a sql query to answer the question
print("Answer:")
print(llm.generate(prompt, max_new_tokens=200))

Question:
 Who is the highest paid NBA player?
Answer:
To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
WHERE SALARY!= '--'
ORDER BY CAST(SALARY AS REAL) DESC
LIMIT 1;
```

This query first filters out the rows where the salary is '--' (i.e., the players who don't have a salary listed). Then, it orders the remaining rows by the salary in descending order (highest to lowest). Finally, it returns the top row, which corresponds to the highest paid NBA player.


<div style="border: 2px solid #009fe3;  margin: 8px; padding: 16px; width: 80%;"> <b>NOTE</b> 

`make_llama_3_prompt` and `get_schema` are commonly used throughout this notebook. Let's inspect them for a second

```python
def make_llama_3_prompt(user, system=""):
    system_prompt = ""
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
```

Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt. 
You can find the documentation on this [model card](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-instruct).

```python
def get_schema():
    return """\
0|Team|TEXT eg. "Toronto Raptors"
1|NAME|TEXT eg. "Otto Porter Jr."
2|Jersey|TEXT eg. "0" and when null has a value "NA"
3|POS|TEXT eg. "PF"
4|AGE|INT eg. "22" in years
5|HT|TEXT eg. `6' 7"` or `6' 10"`
6|WT|TEXT eg. "232 lbs" 
7|COLLEGE|TEXT eg. "Michigan" and when null has a value "--"
8|SALARY|TEXT eg. "$9,945,830" and when null has a value "--"
"""
```
This `get_schema` function returns a description of the `nba_roster` table which you use to inform the model what the datatypes of the columns are (all TEXT) and provide some examples for each column. 

This helps the model know how exactly columns are formatted. 

For example, the `HT` column is formatted `6' 7"` as opposed to `6'7"`. This distinction is important because you may need to `CAST` this column to numerical types in order to do comparison, search, and other mathematical operations on this column. 
</div>

As you can see, this first script will run Llama 3 with prompt tuning to generate SQL queries that are relevant to this database. One thing you may notice is that the response is verbose, we'd have to parse out the sql from the model output.
Let's double check the sqlite query itself.

In [6]:
!sqlite3 nba_roster.db "SELECT NAME, SALARY FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SALARY AS REAL) DESC LIMIT 1;"

Saddiq Bey|$4,556,983


Hey this is incorrect! Evaluating Llama 3 manually by hand will take too much time. We can start automating this process. The correct query is

```sql
SELECT salary, name 
FROM nba_roster
WHERE salary != '--'
ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC
LIMIT 1;
```

In [7]:
!sqlite3 nba_roster.db "SELECT salary, name FROM nba_roster WHERE salary != '--' ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC LIMIT 1;"

$51,915,615|Stephen Curry


## Create an Evaluation Dataset

An Evaluation Dataset is a representative dataset you can use to make sure your model is consistently performing. It can start as few as 20-100 datapoints. The goal is to get started quickly on improving your model, and not get bogged down here.

Here, you can use the example dataset about the nba_roster database at `data/gold-test-set.jsonl`.

<div style="border: 2px solid #009fe3;  margin: 8px; padding: 16px; width: 80%;"> <b>NOTE</b> 

You can do it! Writing an initial evaluation dataset can feel tedious, but a minor investment in time can lead to drastic improvement in quality. In reality, this time investment is going to be made by an LLM user throughout the lifecycle of a model. For some rough time estimates, it took me ~20 minutes to write 20 queries, and that led to a jump in accuracy from 25% to 75%. Later in this notebook, a more intense ~1 hr long data cleaning workflow improved the model accuracy from 75% to 95%.
</div>

## Evaluate the SQL LLM with an Eval LLM 

Next, let's evaluate Llama 3's baseline accuracy for text-to-SQL. Here, we are using a Lamini Inference pipeline. Just as above, you'll see how the output of the model is used to query the SQL database.

First, define a `QueryStage` and `ScoreStage` by extending the `GenerationNode` class.

In [8]:
class QueryStage(GenerationNode):
    def __init__(self, model_name):
        super().__init__(
            model_name=model_name,
            max_new_tokens=150,
        )

    def generate(
        self,
        prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
        *args,
        **kwargs,
    ):
        results = super().generate(
            prompt,
            output_type={"sqlite_query": "str"},
            *args,
            **kwargs,
        )
        return results


    def postprocess(self, obj: PromptObject):
        # Run both the generated and reference (Gold Dataset) SQL queries
        # Assessing whether the SQL queries succeeded in hitting the database (not correctness yet!)
        
        query_succeeded = False

        try:
            logger.info(f"Running SQL query '{obj.response['sqlite_query']}'")
            obj.data["generated_query"] = obj.response["sqlite_query"]
            df = pd.read_sql(obj.response["sqlite_query"], con=engine)
            obj.data['df'] = df
            logger.info(f"Got data: {df}")
            query_succeeded = True

        except Exception as e:
            logger.error(
                f"Failed to run SQL query: {obj.response['sqlite_query']}"
            )

        logger.info(f"Running reference SQL query '{obj.data['sql']}'")
        df = pd.read_sql(obj.data["sql"], con=engine)
        logger.info(f"Got data: {df}")
        obj.data['reference_df'] = df

        logger.info(f"For question: {obj.data['question']}")
        logger.info(f"For query: {obj.response['sqlite_query']}")

        obj.data["query_succeeded"] = query_succeeded

    def preprocess(self, obj: PromptObject):
        new_prompt = make_llama_3_prompt(**self.make_prompt(obj.data))
        obj.prompt = new_prompt

    def make_prompt(self, data: dict):
        system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
        system += "Consider the nba_roster table with the following schema:\n"
        system += get_schema() + "\n"
        system += (
            "Write a sqlite SQL query that would help you answer the following question:\n"
        )
        user = data["question"]
        return {
            "user": user,
            "system": system,
        }

In [9]:
class ScoreStage(GenerationNode):
    def __init__(self):
        super().__init__(
            model_name="meta-llama/Meta-Llama-3-8B-Instruct",
            max_new_tokens=150,
        )

    def generate(
        self,
        prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
        *args,
        **kwargs,
    ):
        results = super().generate(
            prompt,
            output_type={"explanation": "str", "similar": "bool"},
            *args,
            **kwargs,
        )
        return results

    def preprocess(self, obj: PromptObject):
        obj.prompt = make_llama_3_prompt(**self.make_prompt(obj))
        logger.info(f"Scoring Stage Prompt:\n{obj.prompt}")

    def postprocess(self, obj: PromptObject):
        obj.data['is_matching'] = self.is_matching(obj.data, obj.response)
        obj.data['explanation'] = obj.response["explanation"]
        obj.data['similar'] = obj.response["similar"]

    def is_matching(self, data, response):
        return (str(data.get('df',"None")).lower() == str(data['reference_df']).lower() 
                or response['similar'])

    def make_prompt(self, obj: PromptObject):
        # Your evaluation model compares SQL output from the generated and reference SQL queries, using another LLM in the pipeline
        system_prompt = "Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster dataset"
        system_prompt += "Respond with valid JSON {'explanation' : str, 'similar' : bool}"
        user_prompt = (
            f"========== Dataframe 1 =========\n{str(obj.data.get('df','None')).lower()}\n\n"
        )
        user_prompt += (
            f"========== Dataframe 2 =========\n{str(obj.data['reference_df']).lower()}\n\n"
        )
        user_prompt += f"Can you tell me if these dataframes are similar?"
        return {
            "system": system_prompt,
            "user": user_prompt
        }

With these stages, you can define an evaluation pipeline using the `Generation Pipeline` class. In this pipeline, you can indicate that one stage feeds into the next by passing the output of the query stage into the input of the score stage in the `forward` function.

It's important that the input to the evaluation pipeline's `call` function be an iterable over instances of `PromptObject`. You'll be using these objects to store data as it passes through the pipeline.

In [10]:
async def run_eval(dataset, args):

    results = await run_evaluation_pipeline(dataset, args)

    print("Total results:", len(results))

    return results


async def run_evaluation_pipeline(dataset, args):
    results = EvaluationPipeline(args).call(dataset)

    result_list = []

    pbar = tqdm(desc="Saving results", unit=" results")
    async for result in results:
        result_list.append(result)
        pbar.update()
    return result_list


class EvaluationPipeline(GenerationPipeline):
    def __init__(self, args):
        super().__init__()
        self.query_stage = QueryStage(args.sql_model_name)
        self.score_stage = ScoreStage()


    def forward(self, x):
        x = self.query_stage(x)
        x = self.score_stage(x)
        return x


In [11]:
def load_gold_dataset(args):
    path = f"data/{args.gold_file_name}"

    with jsonlines.open(path) as reader:
        for index, obj in enumerate(reversed(list(reader))):
            if index >= args.max_examples:
                break
            yield PromptObject(prompt="", data=obj)

You'll need to save your results somewhere! In this notebook, you can use the `data/results` directory to log a record of your eval experiments. 

It's important to keep track of these experiments. To do this, you can log basic statistics, as well as errors and successes when the model is able to produce SQL which answers the question.

In [12]:
def save_eval_results(results, args):
    base_path = "./data/results"
    now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    experiment_name = f"nba_sql_pipeline_{now}"
    experiment_dir = os.path.join(base_path, experiment_name)
    os.makedirs(os.path.join(base_path, experiment_name))

    # Write args to file
    args_file_name = f"{experiment_dir}/args.txt"
    with open(args_file_name, "w") as writer:
        pprint(args.__dict__, writer)


    def is_correct(r):
        if (
            (result.data["query_succeeded"] and result.data['is_matching']) or 
            result.data["generated_query"] == result.data['sql']
        ):
            return True
        return False

    # Write sql results and errors to file
    results_file_name = f"{experiment_dir}/sql_results.jsonl"
    with jsonlines.open(results_file_name, "w") as writer:
        for result in results:
            if not is_correct(result):
                continue
            writer.write(
                {
                    "question": result.data['question'],
                    "query": result.data["generated_query"],
                    "query_succeeded": result.data["query_succeeded"],
                    "reference_sql": result.data['sql'],
                    "df": str(result.data.get('df', 'None')),
                    "reference_df": str(result.data['reference_df']),
                    'is_matching': result.data['is_matching'],
                    'similar': result.data['similar'],
                }
            )

    results_file_name = f"{experiment_dir}/sql_errors.jsonl"
    with jsonlines.open(results_file_name, "w") as writer:
        for result in results:
            if is_correct(result):
                continue
            writer.write(
                {
                    "question": result.data['question'],
                    "query": result.data["generated_query"],
                    "query_succeeded": result.data["query_succeeded"],
                    "df": str(result.data.get('df', 'None')),
                    "reference_df": str(result.data['reference_df']),
                    'is_matching': result.data['is_matching'],
                    'similar': result.data['similar'],
                }
            )

    # Write statistics to file
    average_sql_succeeded = sum(
        [result.data["query_succeeded"] for result in results]
    ) / len(results)
    average_correct = sum(
        [result.data["query_succeeded"] and result.data['is_matching'] for result in results]
    ) / len(results)

    file_name = f"{experiment_dir}/summary.txt"
    with open(file_name, "w") as writer:
        print(f"Total size of eval dataset: {len(results)}", file=writer)
        print(f"Total size of eval dataset: {len(results)}")
        print(f"Percent Valid SQL Syntax: {average_sql_succeeded*100}", file=writer)
        print(f"Percent Valid SQL Syntax: {average_sql_succeeded*100}")
        print(f"Percent Correct SQL Query: {average_correct*100}", file=writer)
        print(f"Percent Correct SQL Query: {average_correct*100}")



Now, run eval on Llama 3 and see how it does on your evaluation dataset!

In [13]:
args = Args()
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

Saving results: 0 results [00:00, ? results/s]2024-06-21 14:08:35,116 [ERROR] Failed to run SQL query: SELECT POS, MAX(CAST(SUBSTR(SALARY, 2) AS INTEGER) AS Salary FROM nba_roster WHERE SALARY!= '--' GROUP BY POS
2024-06-21 14:08:35,120 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTRING(HT, 0, INSTR(HT,'')-1) AS INTEGER) FROM nba_roster WHERE HT IS NOT NULL
2024-06-21 14:08:35,123 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(HT, 0, INSTR(HT,'')-1) AS INTEGER) FROM nba_roster WHERE HT IS NOT NULL
2024-06-21 14:08:35,125 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(SALARY, 2) AS INTEGER) AS average_salary FROM nba_roster WHERE POS = 'PF' AND SALARY!= '--';
2024-06-21 14:08:40,776 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) AS weight FROM nba_roster WHERE WT IS NOT NULL
2024-06-21 14:08:40,780 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT!= 'NA';

Total results: 20
Total size of eval dataset: 20
Percent Valid SQL Syntax: 55.00000000000001
Percent Correct SQL Query: 30.0


You can view the results in the `data/results` directory, where there's a saved folder with the experiment arguments and results. 

You can see that Llama 3 can answer correctly `30%` of the time on the gold dataset. Additionally, Llama 3 can provide valid sql syntax as an answer `55%` of the time on the gold dataset. 

## Generate Tuning Data with Data LLMs

You might be thinking, "I'd like to do a little better!" - so the next step is Lamini Memory Tuning. 

First, you need tuning data. Let's use Llama 3 to generate some tuning data! You want `question` and `sql` datapoints to help tune the model to generate SQL about the `nba_roster` dataset. The trick here is to work backwards in a pipeline (generate SQL from the schema, then questions from the generated SQL) and to constrain the prompts, so that the generations are more likely to be correct.

You can do this using the following pipeline script.

In [14]:
class ModelStage(GenerationNode):
    def __init__(self):
        super().__init__(
            model_name="meta-llama/Meta-Llama-3-8B-Instruct",
            max_new_tokens=300,
        )

    def generate(
        self,
        prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
        *args,
        **kwargs,
    ):
        prompt = self.add_template(prompt)

        results = super().generate(
            prompt,
            output_type={
                "explanation": "str",
                "sql_query_1": "str",
                "sql_query_2": "str",
            },
            *args,
            **kwargs,
        )

        return results

    async def add_template(self, prompts):
        async for prompt in prompts:
            new_prompt = make_llama_3_prompt(**self.make_prompt(prompt.data))
            yield PromptObject(prompt=new_prompt, data=prompt.data)

    async def process_results(self, results):
        async for result in results:
            if result is None:
                continue

            if result.response is None:
                continue

            logger.info("=====================================")
            logger.info(f"Generted query 1: {result.response['sql_query_1']}")
            logger.info(f"Generted query 2: {result.response['sql_query_2']}")
            logger.info("=====================================")

            if self.check_sql_query(result.response["sql_query_1"]):
                new_result = PromptObject(prompt="", data=copy.deepcopy(result.data))
                new_result.data.generated_sql_query = result.response["sql_query_1"]
                yield new_result

            if self.check_sql_query(result.response["sql_query_2"]):
                new_result = PromptObject(prompt="", data=copy.deepcopy(result.data))
                new_result.data.generated_sql_query = result.response["sql_query_2"]
                yield new_result

    def make_prompt(self, data):
        system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
        system += (
            "Consider a table called 'nba_roster' with the following schema (columns)\n"
        )
        system += get_schema()
        system += "Consider the following questions, and queries used to answer them:\n"
        for example in data.sample:
            system += "Question: " + example["question"] + "\n"
            system += "Query: " + example["sql"] + "\n"

        # Important: generate relevant queries to your reference data
        # Ideally, close to those that are failing so you can show the model examples of how to do it right!
        user = "Write two queries that are similar but different to those above.\n"
        user += "Format the queries as a JSON object, i.e.\n"
        user += '{ "explanation": str, "sql_query_1" : str, "sql_query_2": str }.\n'

        # Next, use Chain of Thought (CoT) and prompt-engineering to help with generating SQL queries
        user += "First write an explanation of why you decided to write these new queries in about 3-5 sentences, then write valid sqlite SQL queries for each of the 2 new queries. Make sure each query is complete and ends with a ;\n"

        return {"system": system, "user": user}

    def check_sql_query(self, query):
        try:
            pd.read_sql(query, con=engine)
        except Exception as e:
            logger.debug(f"Error in SQL query: {e}")
            return False

        logger.info(f"SQL query {query} is valid")

        return True

Saving results: 20 results [00:13,  1.48 results/s]


In [15]:
class QuestionStage(GenerationNode):
    def __init__(self):
        super().__init__(
            model_name="meta-llama/Meta-Llama-3-8B-Instruct",
            max_new_tokens=150,
        )

    def generate(
        self,
        prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
        *args,
        **kwargs,
    ):
        results = super().generate(
            prompt,
            output_type={
                "explanation": "str",
                "question": "str",
            },
            *args,
            **kwargs,
        )
        return results

    def preprocess(self, obj: PromptObject):
        new_prompt = make_llama_3_prompt(**self.make_question_prompt(obj.data))
        obj.prompt = new_prompt

    def make_question_prompt(self, data):
        system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
        system += (
            "Consider a table called 'nba_roster' with the following schema (columns)\n"
        )
        system += get_schema() + "\n"
        system += "Queries, and questions that they are used to answer:\n"
        for example in data.sample:
            system += "Query: " + example["sql"] + "\n"
            system += "Question: " + example["question"] + "\n"

        user = "Now consider the following query.\n"
        user += "Query: " + data.generated_sql_query + "\n"
        user += "Write a question that this query could be used to answer.\n"

        # Using Chain of Thought (CoT) again
        # This time you can do it programmatically with function calling, so you can easily extract a question out of the JSON object
        user += "Format your response as a JSON object, i.e.\n"
        user += '{ "explanation": str, "question": str }.\n'

        user += "First write an explanation in about 3-5 sentences, then write a one sentence question.\n"

        return {"system": system, "user": user}


You can define a new pipeline to generate queries. This one also has multiple stages, and as mentioned above, the trick is that you are working backwards. The first stage writes SQL, which is pertinent to `nba_roster`. You're using prompt tuning to get queries that may be inspired by a sample of our gold dataset—that way, you're getting examples that are relevant to the evaluation (ideally, showing correct examples similar to those that were previously incorrect). Then, you use the question stage to inspect those queries and generate a question that can be answered by the generated query. 

Since the point is to create an model that can move forwards (generate), working backwards like this is just one creative method for data generation that can help constrain the prompts and produce more accurate generated data for tuning. 

In [16]:
async def run_query_gen_pipeline(seed_queries):
    return QueryGenPipeline().call(seed_queries)


class QueryGenPipeline(GenerationPipeline):
    def __init__(self):
        super().__init__()
        self.model_stage = ModelStage()
        self.question_stage = QuestionStage()

    def forward(self, x):
        x = self.model_stage(x)
        x = self.question_stage(x)
        return x


In [17]:
all_examples = []


async def load_seed_queries(args):
    path = f"data/{args.gold_file_name}"

    with jsonlines.open(path) as reader:
        global all_examples

        all_examples = [obj for obj in reader]

    sample_count = args.num_to_generate
    sample_size = 3

    random.seed(42)

    for i in range(sample_count):
        example_sample = ExampleSample(random.sample(all_examples, sample_size), i)

        yield PromptObject(prompt="", data=example_sample)


class ExampleSample:
    def __init__(self, sample, index):
        self.sample = sample
        self.index = index

In [18]:
async def save_generation_results(results, args):
    path = f"data/training_data/{args.training_file_name}"

    pbar = tqdm(desc="Saving results", unit=" results")
    with jsonlines.open(path, "a") as writer:

        async for result in results:
            writer.write(
                {
                    "question": result.response["question"],
                    "sql": result.data.generated_sql_query,
                }
            )
            pbar.update()

        for example in all_examples:
            writer.write(example)
            pbar.update()

In [19]:
args = Args()
seed_queries = load_seed_queries(args)
results = await run_query_gen_pipeline(seed_queries)
await save_generation_results(results, args)

Saving results: 6 results [00:22,  3.31s/ results]

Take a minute to look over the generated data. You may notice that some of the datapoints are incorrect - the SQL is invalid, the questions are duplicated, or the questions may be irrelevant. Let's continue onwards for now - but we'll return to (programmatically) clean the data later!

## Tune Llama 3 with Lamini Memory Tuning

Now it's time to tune Llama 3 with Lamini! You still want to use the Llama 3 template, so you can stream your training data with this in mind.

In [20]:
def make_question(obj):
    system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
    system += "Consider the nba_roster table with the following schema:\n"
    system += get_schema() + "\n"
    system += (
        "Write a sqlite SQL query that would help you answer the following question:\n"
    )
    user = obj["question"]
    return {"system": system, "user": user}

You can submit your data to Lamini Tuning easily. The best defaults for the top LLMs like Llama 3 have been optimized for you.

In [21]:
args = Args()
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

dataset = get_dataset(args, make_question)
finetune_args = get_default_finetune_args()

# Uncomment to train
# llm.train(
#     data_or_dataset_id=dataset,
#     finetune_args=finetune_args,
#     is_public=True,  # For sharing
# )

Saving results: 30 results [00:22,  1.35 results/s]



Uploading data....
Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: 9d3e7264d1b5f24e8aaa60296517b638b157c7e6ef098582adaf90d280694d6e . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='9d3e7264d1b5f24e8aaa60296517b638b157c7e6ef098582adaf90d280694d6e')
Training job submitted! Check status of job 7502 here: https://app.lamini.ai/train/7502


{'job_id': 7502,
 'status': 'SCHEDULED',
 'dataset_id': '9d3e7264d1b5f24e8aaa60296517b638b157c7e6ef098582adaf90d280694d6e'}

<div style="border: 2px solid #009fe3;  margin: 8px; padding: 16px; width: 80%;"> <b>NOTE</b> 

Tuning jobs are queued immediately after you run the above cell! Once they begin, the estimated time is 30 minutes. You can continue in this notebook by using the four pre-prepared models provided in this notebook which we tuned for your convenience. 

When your training job finishes, you can query the newly trained model by 
1. Finding the model id at `https://app.lamini.ai/train`
2. Instantiating a model client with `llm = lamini.Lamini(model_name="<YOUR_MODEL_ID>")`

Training jobs can fail! If it does, try resubmitting your job by re-running the training cell.

</div>

After you submit a job, you can monitor the job status at https://app.lamini.ai/train. There you'll have access to the interface shown below which will help you track jobs, view logs, and get the model ID once training is complete. 

<img src="assets/website.png" alt="Lamini Train Website" width="80%">

Tuning a model takes many attempts and iterations on the generated data, by re-running evaluation and sifting through the results to adjust the data generation pipeline to cover what's still missing. 

Sometimes, those adjustments are incredibly minute—just like in prompt-engineering, it's hard to predict what those adjustments might be, so being able to quickly iterate using your evaluation pipeline and inspecting the results quickly is absolutely key.

That's why Lamini's high-performance inference engine is built to optimize processes for both evaluation and data generation, and then unify them with tuning effectively.

Just for a gauge of what's normal: in the creation of this notebook, over 20 models were tuned. So don't get discouraged if it's not top notch on your first try: the point is actually to build that muscle of iteration—that's the most important piece towards getting the best results.

You'll see one of the iterations in the following sections, to get a feel for what the workflow is like.

Here's a prepared tuned model, so you don't have to wait for the tuning to complete. This notebook has four prepared models for each of the four times we will tune. 

First, go ahead and ask the tuned model a question! 

In [22]:
# You can replace model_name with your model_id when it's ready!
llm = lamini.Lamini(model_name="a5ebf1c4879569101f32444afae5adcafbfce9c5a6ed13035fd892147f7d59bc")

question = """Who is the highest paid NBA player?"""
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""
prompt = make_llama_3_prompt(question, system)
print("Question:\n", question)

# Ask the model to generate a sql query to answer the question
print("Answer:")
print(llm.generate(prompt, max_new_tokens=200))

Question:
 Who is the highest paid NBA player?
Answer:
select salary, name from nba_roster where SALARY!= '--' ORDER BY CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER) DESC LIMIT 1


Much better! You can check against the database that this is correct.

In [23]:
!sqlite3 nba_roster.db "select salary, name from nba_roster where SALARY!= '--' ORDER BY CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER) DESC LIMIT 1;"

$51,915,615|Stephen Curry


## Evaluate the tuned Llama 3

To compare how results have improved quantitatively, rerun the SQL pipeline with the tuned model:

In [24]:
# You can replace sql_model_name with your model_id when it's ready!
args = Args(sql_model_name="a5ebf1c4879569101f32444afae5adcafbfce9c5a6ed13035fd892147f7d59bc")
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

Saving results: 0 results [00:00, ? results/s]2024-06-21 14:09:34,226 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER) FROM nba_roster WHERE WT!= 'NA') as median
Saving results: 20 results [00:26,  1.31s/ results]

Total results: 20
Total size of eval dataset: 20
Percent Valid SQL Syntax: 95.0
Percent Correct SQL Query: 75.0


You can see that the tuned model has 75% correct SQL (compared to 30% for base Llama 3). Bam!

Let's take a look at the `sql_errors.jsonl` file to try and figure out what the model is getting wrong. Here is the error analysis part, which is figuring out what types of errors are occurring. You find that there are 3 types of errors:

<div style="border: 2px solid #eed202;  margin: 8px; padding: 16px; width: 80%;"> <b>Error 1: The tuned model does not filter for null salaries
</b> 

`"What is the average salary of Power Forward players in the NBA"`

```sql
SELECT AVG(CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as average_salary FROM nba_roster WHERE POS='PF' AND SALARY!= '--';

12355651.6714286
```
Reference: 
```sql
select avg(CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as average_salary from nba_roster where POS = 'PF';

10948045.7848101
```
</div>
<div style="border: 2px solid #eed202;  margin: 8px; padding: 16px; width: 80%;"> <b>Error 2: The tuned model incorrectly orders by desc when calculating percentile or omits the offset correction
</b> 

`"What is the 75th percentile salary in the NBA?"` 
 `"What is the 25th percentile salary in the NBA?"` 
 `"What is the 99th percentile salary in the NBA?"`

```sql
SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as salary FROM nba_roster WHERE SALARY!= '--' ORDER BY salary DESC LIMIT 1 OFFSET (SELECT COUNT(*) FROM nba_roster WHERE SALARY!= '--')*75/100-1;

2421720
```
Reference: 
```sql
SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*75/100-1;

13932008
```
</div>
<div style="border: 2px solid #eed202;  margin: 8px; padding: 16px; width: 80%;"> <b>Error 3: The tuned model incorrectly used Average instead of median 
</b> 

`"What's the median age of the Miami Heat?"`

```sql
SELECT AVG(AGE) FROM nba_roster WHERE team='Miami Heat';
```

Reference:
```sql
select CAST(AGE as INTEGER) as percentile from nba_roster where team='Miami Heat' order by percentile limit 1 offset (select count(*) from nba_roster where team='Miami Heat')/2;
```
</div>

## Improve the Tuned Llama 3

You can improve the tuned model by improving the dataset you used based on your error analysis above. To do this, you can both increase the size, coverage, and quality of your generated dataset.

This next step will generate 10x more data. This dataset will still have quality issues, so actually playing a numbers game can help you: generating more data overall means you can filter bad examples from the dataset later and still have a hefty amount of data left.

In [25]:
# If you'd like to generate more data, change num_to_generate, this cell will take longer to run!
args = Args(gold_file_name='gold-test-set.jsonl', training_file_name="generated_queries_large.jsonl", num_to_generate=10)
seed_queries = load_seed_queries(args)
results = await run_query_gen_pipeline(seed_queries)
await save_generation_results(results, args)

Saving results: 6 results [00:36,  4.94s/ results]

Here's another piece of error analysis in your data generation pipeline. After sifting through the data, one thing that stands out is that some queries and questions are duplicated, and some queries may not run.

Here are a few improvements you can easily do — programmatically:

1. Filter the dataset by removing duplicates
2. Only keeping queries that are valid sql.
3. Remove queries where we filter by "Null"
4. Returns an empty dataframe
5. Uses incorrect query components like "AVG(HT)" in the query
6. Add a semicolon to the end if it does not exist

In [26]:
question_set = set()
sql_set = set()

def is_not_valid_sql(question, sql):
    try:
        df = pd.read_sql(sql, con=engine)
        return False
    except Exception as e:
        return True

def has_null_in_sql_or_question(question, sql):
    return "null" in sql.lower() or "null" in question

def returns_empty_dataframe(question, sql):
    try:
        df = pd.read_sql(sql, con=engine)
        return "Empty" in str(df) or "None" in str(df)
    except Exception as e:
        return False
    
def uses_avg_on_ht_column(question, sql):
    return "avg(ht)" in sql.lower() or "avg(salary" in sql.lower() 

filter_conditions = [is_not_valid_sql, has_null_in_sql_or_question, returns_empty_dataframe, uses_avg_on_ht_column]

def training_semicolon(sql):
    if sql.strip()[-1] != ";":
        return sql.strip() + ";"
    return sql

with jsonlines.open("data/training_data/generated_queries_large.jsonl", "r") as reader:
    with jsonlines.open("data/training_data/generated_queries_large_filtered.jsonl", "w") as writer:
        for r in reader:
            if r["question"] in question_set or r["sql"] in sql_set:
                continue
            question_set.add(r["question"])
            sql_set.add(r["sql"])
            
            if any(c(r['question'], r['sql']) for c in filter_conditions):
                continue

            sql = training_semicolon(r['sql'])
            writer.write(
                {
                    "question": r["question"],
                    "sql": sql,
                }
            )

Saving results: 30 results [00:36,  1.20s/ results]


Great! The large 1000 datapoint dataset is filtered down to 364 datapoints. This makes it way easier for the next step of sifting through the data a second time, this time more closely. You'll notice that it's the combination of analyzing and categorizing errors, with building automated pipelines to address those errors that will serve you best. It's important to dive-deep analyses of your data when tuning models, so you can reveal issues that are very difficult to detect on the surface automatically—what's helpful, however, is that you can build out reusable automated pipelines from that, which you can re-run in future iterations of model improvement, when you upgrade your base model (e.g. to Llama 4!), and even when you develop similar adjacent model applications.

Here's what a simple manual look-over as a next step can look like:
1. Print out the SQL queries and questions for easy reading
2. Manually delete or fix obviously incorrect datapoints as you look over each datapoint

In [52]:
limit = 10
with jsonlines.open("data/training_data/generated_queries_large_filtered.jsonl", "r") as reader:
    for i, r in enumerate(reader):
        print(f"===================== {i+1} ======================")
        print(r['question'])        
        print(r['sql'])
        df = pd.read_sql(r['sql'], con=engine)
        print(tabulate(df, headers='keys', tablefmt='sqlite'))
        limit -= 1
        if limit < 0: # Remove this limit if you'd like to pretty print all the data
            break

===================== 1 ======================
What college has the most players in the NBA who are 30 years old or older
SELECT COLLEGE, COUNT(*) AS count FROM nba_roster WHERE AGE >= 30 GROUP BY COLLEGE ORDER BY count DESC LIMIT 1;
    COLLEGE      count
--  ---------  -------
 0  --              22
===================== 2 ======================
What is the total salary of all NBA players
SELECT SUM(CAST(SUBSTR(SALARY, 1, INSTR(SALARY, '$')-1) AS INTEGER)*1000000) FROM nba_roster;
      SUM(CAST(SUBSTR(SALARY, 1, INSTR(SALARY, '$')-1) AS INTEGER)*1000000)
--  -----------------------------------------------------------------------
 0                                                                        0
===================== 3 ======================
What are the most common positions in the NBA
SELECT POS, COUNT(*) AS num_players FROM nba_roster GROUP BY POS;
    POS      num_players
--  -----  -------------
 0  C                 81
 1  F                 95
 2  G                 96


<div style="border: 2px solid #009fe3;  margin: 8px; padding: 16px; width: 80%;"> <b>NOTE</b> 

This step can take time to do, for example an hour filtering through ~350 datapoints. VSCode had a view for the output, which you can get to by clicking into the "..." inside the output cell.

What you're looking for are obviously incorrect datapoints to quickly remove.

You are also scanning for interesting datapoints you had not thought to include in the Gold Dataset.

One hack was to reverse the order of inspection and start at the bottom of the file so you could keep the numbers relevant.

Here's an example datapoint which is incorrect upon inspection:

```bash

===================== 345 ======================
What is the average age of the tallest players in the NBA
SELECT NAME, TEAM, POS, AVG(AGE) AS AVG_AGE FROM nba_roster WHERE CAST(SUBSTR(HT, 1, INSTR(HT,' ')-1) AS INTEGER) + CAST(SUBSTR(HT, INSTR(HT,' ')+1) AS FLOAT)/12 > 6.67 GROUP BY NAME, TEAM, POS ORDER BY AVG_AGE DESC LIMIT 1;
    NAME          Team                POS      AVG_AGE
--  ------------  ------------------  -----  ---------
 0  LeBron James  Los Angeles Lakers  SF            38
 
```
</div>

<img src="assets/manual_filtering.png" alt="Side By Side Filtering" width="80%">




After doing this, you are left with 220 filtered and cleaned datapoints in a new file created manually `generated_queries_large_filtered_cleaned.jsonl`.

You can use this to tune the next iteration of your model.

In [28]:
args = Args(training_file_name="archive/generated_queries_large_filtered_cleaned.jsonl")
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

dataset = get_dataset(args, make_question)
finetune_args = get_default_finetune_args()

# Uncomment to train
# llm.train(
#     data_or_dataset_id=dataset,
#     finetune_args=finetune_args,
#     is_public=True,  # For sharing
# )


Uploading data....
Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: c133dc220b0cb24627b7064b0c8654b6e069abbf403ca730ce34df306619e704 . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='c133dc220b0cb24627b7064b0c8654b6e069abbf403ca730ce34df306619e704')
Training job submitted! Check status of job 7504 here: https://app.lamini.ai/train/7504


{'job_id': 7504,
 'status': 'SCHEDULED',
 'dataset_id': 'c133dc220b0cb24627b7064b0c8654b6e069abbf403ca730ce34df306619e704'}

### Iteratively tune and improve the tuned Llama 3

In [29]:
# You can replace sql_model_name with your model_id when it's ready!
args = Args(sql_model_name="63fd73a775daf24216b46c680a1e963a8d1e02b21bca43fcea6c26737d2e887e", gold_file_name = "gold-test-set.jsonl")
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

Saving results: 0 results [00:00, ? results/s]2024-06-21 14:10:34,562 [ERROR] Failed to run SQL query: SELECT NAME FROM nba_roster WHERE TEAM='Brooklyn Nets' AND AGE=MAX(AGE);
Saving results: 20 results [00:16,  1.21 results/s]

Total results: 20
Total size of eval dataset: 20
Percent Valid SQL Syntax: 95.0
Percent Correct SQL Query: 90.0


Yay! The new model improved to 90% correct on the gold dataset. You can continue this process, looking over the errors and adding, editing, and filtering better data. You can do this by continuing to build more involved programmatic pipelines and skimming manually to understand patterns in the data—until you are satisfied with the accuracy. 

Accuracy on your Gold Dataset is a function of effort. You can reach near 100% accuracy on the Gold Dataset, for example. Typically, the right move is to have the easiest examples in the Gold Dataset that your best model still gets wrong. 

Once you're satisfied with the results on your Gold Dataset, it's time to make your Gold Dataset harder, and then repeat the process of improving the model again.

# Iterate on the Evaluation Dataset

Now that you've gotten good performance on the original Gold Dataset, it's a good time to expand the dataset to make evaluation harder, and in turn, get your tuned model to become even more capable. The augmented `gold-test-set-v2.jsonl` has a few more handcrafted datapoints looking to add coverage over additional complex queries.

First, on your new Gold Dataset, re-establish a baseline performance of Llama 3 on `gold-test-set-v2.jsonl`.

In [30]:
args = Args(gold_file_name='gold-test-set-v2.jsonl')
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

Saving results: 0 results [00:00, ? results/s]2024-06-21 14:10:42,361 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) AS weight FROM nba_roster WHERE WT IS NOT NULL
2024-06-21 14:10:42,363 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT!= 'NA';
2024-06-21 14:10:42,365 [ERROR] Failed to run SQL query: SELECT PERCENTILE(SALARY, 0.25) FROM nba_roster WHERE SALARY!= '--';
2024-06-21 14:10:42,366 [ERROR] Failed to run SQL query: SELECT PERCENTILE(salary, 0.75) FROM (SELECT CAST(SUBSTR(salary, 2) AS INTEGER) AS salary FROM nba_roster WHERE salary!= '--') AS subquery
2024-06-21 14:10:42,368 [ERROR] Failed to run SQL query: SELECT PERCENTILE(salary, 0.99) FROM nba_roster WHERE salary IS NOT NULL
2024-06-21 14:10:42,504 [ERROR] Failed to run SQL query: SELECT AVG(CAST(SUBSTR(SALARY, 2) AS INTEGER) AS average_salary FROM nba_roster WHERE POS = 'PF' AND SALARY!= '--';
2024-06-21 14:10:47,6

Total results: 40
Total size of eval dataset: 40
Percent Valid SQL Syntax: 62.5
Percent Correct SQL Query: 35.0


Looks like there's plenty of room for improvment! You know how this works now:
1. Generate a new training dataset 
2. Train a model
3. Evaluate

In [31]:
args = Args(gold_file_name='gold-test-set-v2.jsonl', training_file_name="generated_queries_v2.jsonl")
seed_queries = load_seed_queries(args)
results = await run_query_gen_pipeline(seed_queries)
await save_generation_results(results, args)

Saving results: 55 results [00:36,  1.53 results/s]


Like before, go ahead and tune a model using this dataset.

In [32]:
args = Args(training_file_name="generated_queries_v2.jsonl")
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

dataset = get_dataset(args, make_question)
finetune_args = get_default_finetune_args()

# Uncomment to train
# llm.train(
#     data_or_dataset_id=dataset,
#     finetune_args=finetune_args,
#     is_public=True,  # For sharing
# )


Uploading data....
Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: b69739e9dd2cd4e886902c39e31a544a7ee88824f3ef21d02648c6d1f85d8e8c . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='b69739e9dd2cd4e886902c39e31a544a7ee88824f3ef21d02648c6d1f85d8e8c')
Training job submitted! Check status of job 7505 here: https://app.lamini.ai/train/7505


{'job_id': 7505,
 'status': 'SCHEDULED',
 'dataset_id': 'b69739e9dd2cd4e886902c39e31a544a7ee88824f3ef21d02648c6d1f85d8e8c'}

In [33]:
# You can replace sql_model_name with your model_id when it's ready!
args = Args(sql_model_name="2e83542ad6df532dd861ca0d3882cd861c2e5df3cefe5dc1f98f5028069d0e8b", gold_file_name='gold-test-set-v2.jsonl')
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

2024-06-21 14:11:49,387 [ERROR] Failed to run SQL query: SELECT team FROM nba_roster GROUP BY team ORDER BY COUNT(*) AS team_size ASC LIMIT 1;
Saving results: 40 results [01:16,  1.90s/ results]
2024-06-21 14:11:57,590 [ERROR] Failed to run SQL query: SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile order by 1 ASC limit 1 offset (select count(*) from nba_roster where SALARY!= '--')*75/100-1;
Saving results: 40 results [00:25,  1.56 results/s]

Total results: 40
Total size of eval dataset: 40
Percent Valid SQL Syntax: 95.0
Percent Correct SQL Query: 75.0


Like before, it's time for a large data generation and cleaning workflow on Lamini's optimized heavy-inference engine.

In [34]:
args = Args(gold_file_name='gold-test-set-v2.jsonl', training_file_name="generated_queries_v2_large.jsonl")
seed_queries = load_seed_queries(args)
results = await run_query_gen_pipeline(seed_queries)
await save_generation_results(results, args)

Saving results: 11 results [01:01,  3.64s/ results]

In [35]:
with jsonlines.open("data/training_data/generated_queries_v2_large.jsonl", "r") as reader:
    with jsonlines.open("data/training_data/generated_queries_v2_large_filtered.jsonl", "w") as writer:
        for r in reader:
            if r["question"] in question_set or r["sql"] in sql_set:
                continue
            question_set.add(r["question"])
            sql_set.add(r["sql"])
            
            if any(c(r['question'], r['sql']) for c in filter_conditions):
                continue

            sql = training_semicolon(r['sql'])
            writer.write(
                {
                    "question": r["question"],
                    "sql": sql,
                }
            )

Saving results: 55 results [01:02,  1.13s/ results]


In [ ]:
limit = 10
with jsonlines.open("data/training_data/generated_queries_v2_large_filtered.jsonl", "r") as reader:
    for i, r in enumerate(reader):
        print(f"===================== {i+1} ======================")
        print(r['question'])        
        print(r['sql'])
        df = pd.read_sql(r['sql'], con=engine)
        print(tabulate(df, headers='keys', tablefmt='sqlite'))
        limit -= 1
        if limit < 0: # Remove this limit if you'd like to pretty print all the data
            break

In [45]:
args = Args(training_file_name="archive/generated_queries_v2_large_filtered_cleaned.jsonl")
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

dataset = get_dataset(args, make_question)
finetune_args = get_default_finetune_args()

# Uncomment to train
# llm.train(
#     data_or_dataset_id=dataset,
#     finetune_args=finetune_args,
#     is_public=True,  # For sharing
# )


Uploading data....
Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: cda99c9fe2b91b181c556558ca6845da8fd678d8cfc38b7af25fc35060d8c5c8 . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='cda99c9fe2b91b181c556558ca6845da8fd678d8cfc38b7af25fc35060d8c5c8')
Training job submitted! Check status of job 7520 here: https://app.lamini.ai/train/7520


{'job_id': 7520,
 'status': 'SCHEDULED',
 'dataset_id': 'cda99c9fe2b91b181c556558ca6845da8fd678d8cfc38b7af25fc35060d8c5c8'}

### Evaluate the tuned Llama 3 (again)

Now that you've tuned another model, you can finally check and see how your tuning impacted the quality of the SQL output—and compare it quantitatively.

In [38]:
# You can replace sql_model_name with your model_id when it's ready!
args = Args(sql_model_name="3f7e740c0ea2227631a30d293b51564ad1b80727c3768a3b136fbae93170c1e2", gold_file_name='gold-test-set-v2.jsonl')
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

Saving results: 40 results [00:25,  1.57 results/s]

Total results: 40
Total size of eval dataset: 40
Percent Valid SQL Syntax: 100.0
Percent Correct SQL Query: 95.0


You've improved accuracy from 30% to 95% for valid SQL query accuracy by tuning Llama 3! Amazing.

<div style="border: 2px solid #4bb543;  margin: 8px; padding: 16px; width: 80%;"> <h2>Lessons</h2> 

As a realistic overlay, here are details on what it took to create this notebook:
1. Multiple automated and manual filtering and editing passes over the tuning data
2. Iterated on the Gold Dataset by adding datapoints you want the model to have coverage over
3. Many tuning jobs (30+) on different iterations of the tuning data
4. Evaluation pipeline construction and prompt-engineering — to have robust evaluation
5. Error analysis by reading the errors and determining if it's an error in our evaluation pipeline or a model error

All this to say - Lamini Memory Tuning is a highly iterative process, don't be discouraged if it doesn't work the first time! Trust that incremental progress can be made and codified by storing training datasets.

Keep in mind that you can always improve the model - even the archived datasets we hand filtered can be improved for further performance. Time box the process and don't hesitate to move on to the next step!

Shipping the model in production can often gather better feedback and datapoints to incorporate into the next tuning iteration—this makes gathering data more of an automated and you can get data that your users care about but that you wouldn't have thought of in a vacuum. To make it less daunting, "shipping in production" can even start with a limited release to 5 users.

Stay tuned for a follow on notebook where we explore How to build a SQL LLM on Lamini using Llama 3!

[Contact us at Lamini](https://www.lamini.ai/contact) to learn even better techniques for building highly accurate LLM models, as well as running this all in your own VPC or on-premise environments.

</div>